<a href="https://colab.research.google.com/github/AarthiKalyanasundaram/DeepFM_library/blob/master/Predicting_the_Rating_using_DeepFM_and_xDeepFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing DeepFM**

In [7]:
!pip install deepctr

     |████████████████████████████████| 122kB 5.3MB/s 


**Importing the Library**

In [9]:
#Importing required Libraries
import pandas as pd
import torch
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from deepctr.models import DeepFM
from deepctr.feature_column import SparseFeat, VarLenSparseFeat,get_feature_names

**Reading the data**

In [2]:
#reading data using pandas 
from google.colab import files
uploaded = files.upload()
import io
data = pd.read_csv(io.BytesIO(uploaded['transaction_data.csv']))

Saving transaction_data.csv to transaction_data.csv


**Find out the shape of the data**

In [3]:
data.shape

(403046, 6)

In [4]:
sparse_features = ["merchant_id", "customer_id"]
target = ['rating']

**Label Encoder**

In [10]:
for feat in sparse_features:
  lbe = LabelEncoder()
  data[feat] = lbe.fit_transform(data[feat])

**count unique features for each sparse field**

In [11]:
fixlen_feature_columns = [SparseFeat(feat, data[feat].nunique(),embedding_dim=4)for feat in sparse_features]
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

**Input data for model**

In [12]:
train, test = train_test_split(data, test_size=0.2)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

**Model Building**

In [13]:
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='regression')
model.compile("adam", "mse", metrics=['mse'], )

**Fitting, Predicting and Print**

In [17]:
history = model.fit(train_model_input, train[target].values,batch_size=256, epochs=5, verbose=2, validation_split=0.2, )
pred_ans = model.predict(test_model_input, batch_size=256)
print("test MSE", round(mean_squared_error(test[target].values, pred_ans), 4))

Epoch 1/5
1008/1008 - 4s - loss: 0.0099 - mse: 0.0090 - val_loss: 0.0170 - val_mse: 0.0161
Epoch 2/5
1008/1008 - 4s - loss: 0.0097 - mse: 0.0088 - val_loss: 0.0165 - val_mse: 0.0156
Epoch 3/5
1008/1008 - 4s - loss: 0.0095 - mse: 0.0087 - val_loss: 0.0181 - val_mse: 0.0172
Epoch 4/5
1008/1008 - 4s - loss: 0.0094 - mse: 0.0085 - val_loss: 0.0173 - val_mse: 0.0164
Epoch 5/5
1008/1008 - 4s - loss: 0.0092 - mse: 0.0083 - val_loss: 0.0178 - val_mse: 0.0169
test MSE 0.0167


In [18]:
pred_ans

array([[1.0326089],
       [1.0097116],
       [1.0209787],
       ...,
       [1.0059351],
       [1.0287349],
       [1.0153273]], dtype=float32)

**Importing the relevant libraries**

In [22]:
from deepctr.models import xDeepFM

**Model Building**

In [23]:
xDeepFMmodel = xDeepFM(linear_feature_columns, dnn_feature_columns, task='regression')
xDeepFMmodel.compile("adam", "mse", metrics=['mse'], )

**Fitting, Predicting and Evaluating the Model**

In [24]:
historyxDeepFM = xDeepFMmodel.fit(train_model_input, train[target].values,batch_size=256, epochs=5, verbose=2, validation_split=0.2, )
pred_ans_xDeepFM = xDeepFMmodel.predict(test_model_input, batch_size=256)
print("test MSE", round(mean_squared_error(test[target].values, pred_ans_xDeepFM), 4))

Epoch 1/5


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


1008/1008 - 19s - loss: 0.0231 - mse: 0.0230 - val_loss: 0.0150 - val_mse: 0.0148
Epoch 2/5
1008/1008 - 18s - loss: 0.0135 - mse: 0.0132 - val_loss: 0.0149 - val_mse: 0.0145
Epoch 3/5
1008/1008 - 18s - loss: 0.0124 - mse: 0.0119 - val_loss: 0.0156 - val_mse: 0.0151
Epoch 4/5
1008/1008 - 18s - loss: 0.0119 - mse: 0.0114 - val_loss: 0.0155 - val_mse: 0.0150
Epoch 5/5
1008/1008 - 18s - loss: 0.0114 - mse: 0.0108 - val_loss: 0.0165 - val_mse: 0.0158
test MSE 0.0155


In [25]:
pred_ans_xDeepFM

array([[1.0207521],
       [1.0308435],
       [1.0397707],
       ...,
       [1.0077271],
       [1.0207572],
       [1.0522134]], dtype=float32)